In [1]:
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
import pandas as pd
from omegaconf import OmegaConf
import os
import torch
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.gridspec import GridSpec
from tqdm.notebook import tqdm
import bm
# os.chdir(Path(bm.__file__).parent.parent)
from bm import train

%matplotlib inline
%config InlineBackend.figure_format='retina'

Hostname bagration not defined in /conf/study_paths/study_paths.yaml. Using default paths.


In [2]:
output_dir = train.main.dora.dir
print(output_dir)
eval_dir = output_dir / "eval" / "signatures"
sigs_to_eval = [p.name for p in (output_dir / "grids" / "nmi.neuro_experiments").iterdir()]
print(sigs_to_eval)
assert output_dir.exists()
assert eval_dir.exists()

/home/zubrikhina/brainmagick_MICCAI/outputs
[]


In [9]:
os.listdir('/home/zubrikhina/brainmagick_MICCAI/outputs/eval/signatures/')

['bdad740f',
 'b482e844',
 'a718b916',
 '79b08170',
 'e8423e7f',
 'dc9ffb18',
 '4d9b4a40',
 '14feb973',
 '53a95efb',
 'c0c688c8',
 'c97c100b',
 'c9f137c9',
 'f192aae8',
 '663aa93c',
 'e5be6326',
 '342eaad6',
 '884343d2']

In [10]:
# sig4='342eaad6'
sig3='590d7e16'

In [11]:
def load_data_from_sig(sig, level="segment"):
    """
    Load data from solver signature
    - probs (torch tensor): probability on vocab [N, V]
    - vocab (torch tensor): vocab of word hashes [V]
    - words (torch tensor): the word hash for each sample [N]
    - metadata (panda dataframe) of len [N] which contains for each sample:
           'word_hashes', 'word_indices', 'seq_indices',
           'word_strings', 'subject_id', 'recording_id'
    """
    assert level in ["word", "segment"], "level should be 'word' or 'segment'"
    probs = torch.load(eval_dir / sig / f"probs_{level}.pth") 
    vocab = torch.load(eval_dir / sig / f"vocab_{level}.pth") # vocab (hashes)
    metadata = pd.read_csv(eval_dir / sig / "metadata.csv", index_col=0, low_memory=False) 
    
    words = torch.tensor(metadata[f"{level}_hashes"].tolist()).long() # for each sample, the word (hashes)
    assert probs.shape == (len(words), len(vocab)), (probs.shape, len(words), len(vocab))
    assert len(words) == len(metadata)
    metadata["idx"] = range(len(words))
    metadata["word_strings"] = metadata["word_strings"].str.lower()

    return probs, vocab, words, metadata

In [12]:
def get_accuracy_from_probs(probs, row_labels, col_labels, topk=10):
    """
    probs: for each row, the probability distribution over a vocab
    returns the topk accuracy that the topk best predicted labels
    match the row_labels
    Inputs:
        probs: of shape [B, V] probability over vocab, each row sums to 1
        row_labels: of shape [B] true word for each row
        col_labels: [V] word that correspond to each column
        topk: int
    Returns: float scalar, topk accuracy
    """
    assert len(row_labels) == len(probs)
    assert len(col_labels) == probs.shape[1]

    # Extract topk indices
    idx = probs.topk(topk, dim=1).indices

    # Get the corresponding topk labels
    whs = col_labels[idx.view(-1)].reshape(idx.shape)

    # 1 if the labels matches with the targets
    correct = ((whs == row_labels[:, None]).any(1)).float()
    assert len(correct) == len(row_labels)

    # Average across samples
    acc = correct.mean()

    return acc.item()

In [13]:
# probs4, vocab4, words4, metadata4 =load_data_from_sig(sig4, level ='segment')
probs3, vocab3, words3, metadata3 =load_data_from_sig(sig3, level ='segment')

FileNotFoundError: [Errno 2] No such file or directory: '/home/zubrikhina/brainmagick_MICCAI/outputs/eval/signatures/590d7e16/probs_segment.pth'

In [16]:
probs3.shape

torch.Size([69972, 1428])

In [17]:
words3.shape

torch.Size([69972])

In [22]:
get_accuracy_from_probs(probs3,words3, vocab3, topk=10)

0.7029669284820557

In [23]:
get_accuracy_from_probs(probs3,words3, vocab3, topk=1)

0.412322074174881

In [24]:
## уменьшим наш словарь до 1228 сегментов 

In [46]:
def remove_elements_from_vocab(vocab,words,probs, size=200):
    random_indx =np.random.choice(list(range(1428)), size=size, replace=False)
    new_vocab =np.delete(vocab, random_indx)
    elem_to_delete=vocab[random_indx]
    indexes_to_delete =np.argwhere(np.isin(words.numpy(),elem_to_delete))
    new_words=np.delete(words.numpy(), indexes_to_delete)
    new_probs =np.delete(probs.numpy(),indexes_to_delete, axis=0)
    new_probs=np.delete(new_probs,random_indx, axis=1)
    return new_vocab, torch.tensor(new_words), torch.tensor(new_probs)

In [47]:
new_vocab, new_words, new_probs =remove_elements_from_vocab(vocab3,words3,probs3, size=200)

In [48]:
new_vocab.shape

torch.Size([1228])

In [49]:
new_probs.shape

torch.Size([60172, 1228])

In [53]:
60172 /1228

49.0

In [54]:
new_words.shape

torch.Size([60172])

In [55]:
get_accuracy_from_probs(new_probs,new_words, new_vocab, topk=10)

0.7197367548942566

In [52]:
get_accuracy_from_probs(new_probs,new_words, new_vocab, topk=1)

0.42825567722320557

In [16]:
idx_best = probs.topk(5, dim=1).indices

In [17]:
predicted_top5 = vocab[idx_best.view(-1)].reshape(idx_best.shape)

In [18]:
predicted_top5.numpy()[:, 0].shape

(69972,)

In [178]:
metadata['predicted0'] =predicted_top5.numpy()[:, 0]

In [179]:
metadata['predicted1'] =predicted_top5.numpy()[:, 1]

In [180]:
metadata['predicted2'] =predicted_top5.numpy()[:, 2]

In [181]:
metadata['predicted3'] =predicted_top5.numpy()[:, 3]

In [182]:
metadata['predicted4'] =predicted_top5.numpy()[:, 4]

In [183]:
metadata.segment_hashes.value_counts()

segment_hashes
 1023044119420351441    49
-7147995716960060833    49
 8272874790887045404    49
 1572282642160955831    49
 6930124483747195011    49
                        ..
 7402442564258595346    49
-4199343681639433574    49
 2004080571053502143    49
 1489792118339584915    49
 6659854869743850346    49
Name: count, Length: 1428, dtype: int64

In [185]:
metadata.loc[(metadata.segment_hashes==1023044119420351441)].shape

(49, 17)

In [186]:
len(metadata.loc[(metadata.segment_hashes==1023044119420351441)].subject_id.unique())

27

In [187]:
metadata.loc[(metadata.segment_hashes==1023044119420351441) & (metadata.subject_id==8)]

,segment_hashes,word_hashes,word_indices,seq_indices,word_segment_strings,word_strings,subject_id,subject_uid,recording_uid,recording_id,study,idx,predicted0,predicted1,predicted2,predicted3,predicted4
21420,1023044119420351441,86905,1,-5194366708009140224,Results Harmon she suppressed the surge of ann...,harmon,8,08-08-08-08-08-08-08-08-08-08-08-08-08-08-08-0...,08_session1_story3-08_session1_story3-08_sessi...,60,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,21420,5894848821703666756,-4980211862117205046,5813185885679571821,-461628926137136066,-1440649480385077068
22848,1023044119420351441,86905,1,-5194366708009140224,Results Harmon she suppressed the surge of ann...,harmon,8,09-09-09-09-09-09-09-09-09-09-09-09-09-09-09-0...,09_session0_story3-09_session0_story3-09_sessi...,64,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,22848,3138008871906447805,8234152304727384126,1023044119420351441,-8871719643152020172,-7663930208036725935


In [188]:
all_packages =os.listdir("cache/studies/gwilliams2022")

In [189]:
len([pack  for pack in all_packages if "_story3" in pack])

49

In [190]:
len(metadata.subject_id.unique())

27

In [191]:
indx= 0

In [192]:
def get_predicted_results(indx):
    example =metadata.iloc[indx, :]
    
    print("Original word: ", example.word_strings)
    print("Original segment: ", example.word_segment_strings)
    
    print("Predicted: ")
    predicted0 =metadata.loc[metadata.segment_hashes ==example.predicted0]
    predicted1 =metadata.loc[metadata.segment_hashes ==example.predicted1]
    predicted2 =metadata.loc[metadata.segment_hashes ==example.predicted2]
    predicted3 =metadata.loc[metadata.segment_hashes ==example.predicted3]
    predicted4 =metadata.loc[metadata.segment_hashes ==example.predicted4]
    
    predicts =[predicted0,predicted1,predicted2,predicted3, predicted4]
    for i, pred in enumerate(predicts):
        print(f"Predicted word: {i}", pred.iloc[0,:].word_strings)
        print(f"Predicted segment: {i}", pred.iloc[0,:].word_segment_strings)
        print("_______")

In [194]:
for i in range(0,50):
    print(i)
    get_predicted_results(i)
    print("______________")

0
Original word:  harmon
Original segment:  Results Harmon she suppressed the surge of annoyance
Predicted: 
Predicted word: 0 he
Predicted segment: 0 the hill and he led me to little patch of ground with a stick
_______
Predicted word: 1 house
Predicted segment: 1 house as two laborers wrapped a great
_______
Predicted word: 2 put
Predicted segment: 2 deduction magician put cash call realized kiss where well lights open
_______
Predicted word: 3 going
Predicted segment: 3 going to do it in two parts last part first
_______
Predicted word: 4 also
Predicted segment: 4 considered also motivation whether some good
_______
______________
1
Original word:  she
Original segment:  Harmon she suppressed the surge of annoyance that ran
Predicted: 
Predicted word: 0 she
Predicted segment: 0 Harmon she suppressed the surge of annoyance that ran
_______
Predicted word: 1 design
Predicted segment: 1 hell design traffic wont drew high stop highway
_______
Predicted word: 2 raised
Predicted segment: 

In [112]:
metadata.loc[metadata.segment_hashes==metadata.predicted1]

,segment_hashes,word_hashes,word_indices,seq_indices,word_segment_strings,word_strings,subject_id,subject_uid,recording_uid,recording_id,study,idx,predicted0,predicted1,predicted2,predicted3,predicted4
15,-2405479116573718851,95400,18,-8450910692516560896,contemplated the gift of getting all the hot n...,gift,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,15,3869325127004030740,-2405479116573718851,-860557806313723157,-5033280491285988572,2730250912872551664
17,1384768518026339844,3211,20,-8450910692516560896,gift of getting all the hot news first,getting,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,17,6272361534120822197,1384768518026339844,6219378032576322031,8989460904283557442,-4292408275715717267
19,-685024533533335549,64917,22,-8450910692516560896,of getting all the hot news first face,the,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,19,-8513126228172831823,-685024533533335549,1731077993760627170,5837609269005258717,8649345418052716182
42,-25057246052904293,28063,4,-2117268931029762048,It was not an attractive face right now her eb...,attractive,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,42,-2498328081611615920,-25057246052904293,6783931977482939503,-389555999848938794,-7092947398503682675
48,6645134533014673714,8387,10,-2117268931029762048,her ebony eyes by hours on the watch full lips,eyes,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,48,-5078633591881632820,6645134533014673714,60873158911727124,-2990620593256738730,-940397725889811545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60121,7005341683855906654,50838,15,-5200037439229394944,tree and let forth a moan a cry of purest agon...,a,26,27-27-27-27-27-27-27-27-27-27-27-27-27-27-27-2...,27_session1_story3-27_session1_story3-27_sessi...,195,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,60121,-6780876752710051309,7005341683855906654,-2289996790980635307,-4794975001148810544,3529969198857560404
60126,-5394177555110262226,46920,20,-5200037439229394944,moan a cry of purest agony that escaped him as...,purest,26,27-27-27-27-27-27-27-27-27-27-27-27-27-27-27-2...,27_session1_story3-27_session1_story3-27_sessi...,195,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,60126,-2566958507368491683,-5394177555110262226,-7252951931141503598,8415801904038005511,-5456772522120864751
60137,2189295741976013722,64917,31,-5200037439229394944,tears seeped from the corners of his eyes and ...,the,26,27-27-27-27-27-27-27-27-27-27-27-27-27-27-27-2...,27_session1_story3-27_session1_story3-27_sessi...,195,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,60137,-5710491697289470331,2189295741976013722,-2566672544355215277,-613915730187082442,-602262555225774821
60138,8590776278308325079,42282,32,-5200037439229394944,tears seeped from the corners of his eyes and ...,corners,26,27-27-27-27-27-27-27-27-27-27-27-27-27-27-27-2...,27_session1_story3-27_session1_story3-27_sessi...,195,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,60138,-4416339911348354645,8590776278308325079,-4542886774333374466,3501592433725429676,-7112839770829289208


In [113]:
metadata.loc[metadata.segment_hashes ==3869325127004030740]

,segment_hashes,word_hashes,word_indices,seq_indices,word_segment_strings,word_strings,subject_id,subject_uid,recording_uid,recording_id,study,idx,predicted0,predicted1,predicted2,predicted3,predicted4
1146,3869325127004030740,14050,14,-4693595786406002688,up the behind in a great swirling eddy a,behind,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story3-01_session0_story3-01_sessi...,3,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,1146,3869325127004030740,8372297168339981263,1403127518333687493,-6597552637736141889,4908376757749425666
2374,3869325127004030740,14050,14,-4693595786406002688,up the behind in a great swirling eddy a,behind,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session1_story3-01_session1_story3-01_sessi...,7,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,2374,3869325127004030740,2441962701312108745,5590408429089826425,8765359816832157945,3417416585357072749
3602,3869325127004030740,14050,14,-4693595786406002688,up the behind in a great swirling eddy a,behind,1,02-02-02-02-02-02-02-02-02-02-02-02-02-02-02-0...,02_session0_story3-02_session0_story3-02_sessi...,11,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,3602,3869325127004030740,-6417248649664629721,3590067967339288642,4156583985968134938,721834279407299499
4830,3869325127004030740,14050,14,-4693595786406002688,up the behind in a great swirling eddy a,behind,1,02-02-02-02-02-02-02-02-02-02-02-02-02-02-02-0...,02_session1_story3-02_session1_story3-02_sessi...,15,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,4830,3869325127004030740,-4888442196211315768,-6243628171703295386,2157313013381916718,-5151365911325004654
6058,3869325127004030740,14050,14,-4693595786406002688,up the behind in a great swirling eddy a,behind,2,03-03-03-03-03-03-03-03-03-03-03-03-03-03-03-0...,03_session0_story3-03_session0_story3-03_sessi...,19,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,6058,3869325127004030740,1236908565350942323,-3319331884333283856,4661261337575763561,6645134533014673714
7286,3869325127004030740,14050,14,-4693595786406002688,up the behind in a great swirling eddy a,behind,3,04-04-04-04-04-04-04-04-04-04-04-04-04-04-04-0...,04_session0_story3-04_session0_story3-04_sessi...,23,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,7286,3869325127004030740,-7712430957025460359,2082976704866860041,938868422822893341,6734040600667746557
8514,3869325127004030740,14050,14,-4693595786406002688,up the behind in a great swirling eddy a,behind,3,04-04-04-04-04-04-04-04-04-04-04-04-04-04-04-0...,04_session1_story3-04_session1_story3-04_sessi...,27,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,8514,3869325127004030740,-7577633729465591148,-1649921768267582207,223554476225686355,2082976704866860041
9742,3869325127004030740,14050,14,-4693595786406002688,up the behind in a great swirling eddy a,behind,4,05-05-05-05-05-05-05-05-05-05-05-05-05-05-05-0...,05_session0_story3-05_session0_story3-05_sessi...,31,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,9742,3869325127004030740,8372297168339981263,-5245113058760270046,-8190750586067570576,2082976704866860041
10970,3869325127004030740,14050,14,-4693595786406002688,up the behind in a great swirling eddy a,behind,4,05-05-05-05-05-05-05-05-05-05-05-05-05-05-05-0...,05_session1_story3-05_session1_story3-05_sessi...,35,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,10970,-1758598711732197759,5970870974785529511,-487391963684372443,8405691280914957250,8607737871593519215
12198,3869325127004030740,14050,14,-4693595786406002688,up the behind in a great swirling eddy a,behind,5,06-06-06-06-06-06-06-06-06-06-06-06-06-06-06-0...,06_session0_story3-06_session0_story3-06_sessi...,39,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,12198,3869325127004030740,-8642861696809214522,6645134533014673714,6069082311193776720,-7013990878469297436


In [106]:
metadata.loc[metadata.segment_hashes ==-8890531049469941562].word_strings.values[0]

'of'

In [107]:
metadata.loc[metadata.segment_hashes ==-407396914320614433].word_strings.values[0]

'at'

In [108]:
metadata.loc[metadata.segment_hashes ==-6346595967559545544].word_strings.values[0]

'home'

In [109]:
metadata.loc[metadata.segment_hashes ==-5115296624358818020].word_strings.values[0]

'behind'

In [71]:
predicted_top5.shape

torch.Size([60172, 5])

In [34]:
len(vocab)

1228

In [31]:
acc = get_accuracy_from_probs(probs, words, vocab, topk=1)

In [32]:
acc

0.562404453754425

In [30]:
probs.shape

torch.Size([60172, 1228])

In [20]:
len(vocab)

1228

In [14]:
words

tensor([-5084843884704632058, -2314259720181546301,  5193713870775898170,
         ..., -2116367112856494508,  1136117801389969786,
        -1689416738284983303])

In [18]:
len(metadata.segment_hashes.unique())

1228

In [21]:
len(metadata.word_hashes.unique())

622

In [36]:
len(metadata.segment_hashes.unique())

1228

In [42]:
metadata_subj_id1 =metadata.loc[(metadata.subject_id==0)].drop_duplicates(subset=['segment_hashes'])

In [50]:
metadata_subj_id1.loc[metadata_subj_id1.word_strings =='she']

,segment_hashes,word_hashes,word_indices,seq_indices,word_segment_strings,word_strings,subject_id,subject_uid,recording_uid,recording_id,study,idx
1,-2314259720181546301,40897,2,-8450910692516560896,Harmon she suppressed the surge of annoyance t...,she,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,1
12,-1821106628883108731,40897,13,-8450910692516560896,ran through her as she contemplated the gift o...,she,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,12
76,-3526534289331001854,40897,18,4522449104961273856,blonde curls as she away with the approaching ...,she,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,76


In [52]:
metadata_subj_id1.word_segment_strings.values[0]

'Results Harmon she suppressed the surge of annoyance that ran'

In [53]:
metadata_subj_id1.word_segment_strings.values[1]

'Harmon she suppressed the surge of annoyance that ran through her as she'

In [54]:
metadata_subj_id1.word_segment_strings.values[2]

'Harmon she suppressed the surge of annoyance that ran through her as she contemplated'

In [56]:
metadata_subj_id1.word_segment_strings.values[3]

'she suppressed the surge of annoyance that ran through her as she contemplated'

In [57]:
metadata_subj_id1.word_segment_strings.values[4]

'suppressed the surge of annoyance that ran through her as she contemplated'

In [63]:
metadata_subj_id1.word_strings.values[4]

'surge'

In [59]:
metadata_subj_id1.word_segment_strings.values[5]

'suppressed the surge of annoyance that ran through her as she contemplated the'

In [62]:
metadata_subj_id1.word_strings.values[5]

'of'

In [65]:
metadata_subj_id1

,segment_hashes,word_hashes,word_indices,seq_indices,word_segment_strings,word_strings,subject_id,subject_uid,recording_uid,recording_id,study,idx
0,-5084843884704632058,2019,1,-8450910692516560896,Results Harmon she suppressed the surge of ann...,harmon,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,0
1,-2314259720181546301,40897,2,-8450910692516560896,Harmon she suppressed the surge of annoyance t...,she,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,1
2,5193713870775898170,80989,3,-8450910692516560896,Harmon she suppressed the surge of annoyance t...,suppressed,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,2
3,-2283512739493788751,64917,4,-8450910692516560896,she suppressed the surge of annoyance that ran...,the,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,3
4,1429679597612803236,49502,5,-8450910692516560896,suppressed the surge of annoyance that ran thr...,surge,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story0-01_session0_story0-01_sessi...,0,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1223,-5938933776270351594,98194,61,-5200037439229394944,stream flowing through the ground beneath them,through,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story3-01_session0_story3-01_sessi...,3,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,1223
1224,4558849814146413182,64917,62,-5200037439229394944,flowing through the ground beneath them,the,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story3-01_session0_story3-01_sessi...,3,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,1224
1225,-2116367112856494508,66018,63,-5200037439229394944,flowing through the ground beneath them,ground,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story3-01_session0_story3-01_sessi...,3,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,1225
1226,1136117801389969786,92798,64,-5200037439229394944,flowing through the ground beneath them,beneath,0,01-01-01-01-01-01-01-01-01-01-01-01-01-01-01-0...,01_session0_story3-01_session0_story3-01_sessi...,3,gwilliams2022-gwilliams2022-gwilliams2022-gwil...,1226
